In [1]:
from typing import NamedTuple
from collections import deque, Counter
from itertools import combinations

### Part One

The plan is to perform a breadth-first search from the end and note the distance of every point on the map. Points that are not reachable from the end, or are blocks will have a distance of `inf` just to keep things simple. Also save a set of the block locations to make it easy to iterate over them.

In part one, for each block, look at its neighbor points and their distance to the end. If the difference of these distances is greater than two, you can save time, but removing the blocks. This should run in reasonable linear time (not counting the BFS).

### Part Two
This worked, but is slow.
For each path point A, compare it with all points B that are withing a manhattan distance of 20. If the A's distance to the end it longer than walking the manattan distance plus B's distance, then it's a better path and saves the difference. Could be sped up a bit by caching the the comparison since we compare A and B and later compare B and A. 

It seems like the input allows for some sloppiness. There are no edge cases where there is more than one path between start and end going through the same point. IOW - no branching. 🤷 I guess we should take it!



In [2]:
class Point(NamedTuple):
    row: int
    col: int

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)

    def m_dist(self, other):
        '''Manhattan Distance'''
        return abs(self.col - other.col) + abs(self.row - other.row)
    

class Map:
    directions = [Point(0, 1),Point(0, -1),Point(-1, 0),Point(1, 0)] 
    
    def __init__(self, s):
        self.map = s

        self.start = None
        self.end = None
        self.w = len(s[0])
        self.h = len(s)
        self.blocks = set()
        self.distances = {}
        
        for row, line in enumerate(s):
            for col, c in enumerate(line):
                if c == '#':
                    self.blocks.add(Point(row, col))
                if c == 'S':
                    self.start = Point(row, col)
                if c == 'E':
                    self.end = Point(row, col)

    def neighbors(self, point):
        for d in Map.directions:
            p = point + d
            if 0 <= p.row < self.h and 0 <= p.col < self.w and self.map[p.row][p.col] != '#':
                yield p
                        
        
    def bfs(self, start, stop):
        seen = set([start])
        d = deque([(start, 0)])
        dist = None
        while d:
            loc, distance = d.popleft()

            # Keep track of distances of every findable point
            self.distances[loc] = distance
            
            if loc == stop:
                dist =  distance
            
            for n in self.neighbors(loc):
                if n in self.blocks or n in seen:
                    continue
                d.append((n, distance + 1))
                seen.add(n)
        
        return dist
        
    def cheat(self, block):
        '''return savings if the block is removed'''    
        for a, b in (combinations(filter(lambda p: p in m.distances, self.neighbors(block)), r=2)):
            savings = abs(self.distances[a] -  self.distances[b]) - 2
            if savings >= 2: 
                return savings
        return 0

    def points_at_distance(self, point, d):
        # Return all points within a manahattan distance of d from p
        for row in range(max(0, point.row - d), min(self.h, point.row + d + 1)):
            offset = d - abs(row - point.row)
            for col in range(max(0, point.col - offset), min(self.w,  point.col + offset + 1)):
                dest = Point(row, col)
                if dest != point and dest in self.distances:
                    yield Point(row, col)
   
    def cheat2(self):
        # Find point within a 20 step manhattan distance walk 
        # if that point + the walk are closer, it's a better path
        counts = Counter()
        for source, dist in self.distances.items():
            for dest in self.points_at_distance(source, 20):
                m_dist = source.m_dist(dest)
                if dist > self.distances[dest] + m_dist:
                    savings = dist - (self.distances[dest] + m_dist)
                    counts[savings] += 1
                         
        return counts



In [3]:
s='''###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############'''.split('\n')


m = Map(s)
m.bfs(m.end, m.start)

from collections import Counter
counts = Counter()
for b in m.blocks:
    savings =  m.cheat(b)
    if savings:
       counts[savings] += 1

for d, count in sorted(counts.items(), key=lambda i: i[0]):
    print(f"{count} that save {d}")


14 that save 2
14 that save 4
2 that save 6
4 that save 8
2 that save 10
3 that save 12
1 that save 20
1 that save 36
1 that save 38
1 that save 40
1 that save 64


In [4]:
counts2 = m.cheat2()

for d, count in sorted(counts2.items(), key=lambda i: i[0]):
    if d >= 50:
        print(f"{count} that save {d}")

32 that save 50
31 that save 52
29 that save 54
39 that save 56
25 that save 58
23 that save 60
20 that save 62
19 that save 64
12 that save 66
14 that save 68
12 that save 70
22 that save 72
4 that save 74
3 that save 76


In [5]:
with open('input_files/20.txt') as f:
    raw = f.read().splitlines()

m = Map(raw)
m.bfs(m.end, m.start)
count = 0
for b in m.blocks:
    savings = m.cheat(b)
    if savings and savings >= 100:
       count += 1

print("part one:", count)


part one: 1426


In [6]:
counts2 = m.cheat2()

In [7]:
total = 0
for d, count in counts2.items():
    if d >= 100:
        total += count
print("part two:", total)

part two: 1000697


In [11]:
m = Map(s)
m.bfs(m.start, m.end)

for (row, col), dist in m.distances.items():
    u = row + col
    v = row - col
    print(row, col, u, v, dist)

3 1 4 2 0
2 1 3 1 1
1 1 2 0 2
1 2 3 -1 3
1 3 4 -2 4
2 3 5 -1 5
3 3 6 0 6
3 4 7 -1 7
3 5 8 -2 8
2 5 7 -3 9
1 5 6 -4 10
1 6 7 -5 11
1 7 8 -6 12
2 7 9 -5 13
3 7 10 -4 14
4 7 11 -3 15
5 7 12 -2 16
6 7 13 -1 17
7 7 14 0 18
7 8 15 -1 19
7 9 16 -2 20
6 9 15 -3 21
5 9 14 -4 22
4 9 13 -5 23
3 9 12 -6 24
2 9 11 -7 25
1 9 10 -8 26
1 10 11 -9 27
1 11 12 -10 28
1 12 13 -11 29
1 13 14 -12 30
2 13 15 -11 31
3 13 16 -10 32
3 12 15 -9 33
3 11 14 -8 34
4 11 15 -7 35
5 11 16 -6 36
5 12 17 -7 37
5 13 18 -8 38
6 13 19 -7 39
7 13 20 -6 40
7 12 19 -5 41
7 11 18 -4 42
8 11 19 -3 43
9 11 20 -2 44
9 12 21 -3 45
9 13 22 -4 46
10 13 23 -3 47
11 13 24 -2 48
11 12 23 -1 49
11 11 22 0 50
12 11 23 1 51
13 11 24 2 52
13 10 23 3 53
13 9 22 4 54
12 9 21 3 55
11 9 20 2 56
10 9 19 1 57
9 9 18 0 58
9 8 17 1 59
9 7 16 2 60
10 7 17 3 61
11 7 18 4 62
12 7 19 5 63
13 7 20 6 64
13 6 19 7 65
13 5 18 8 66
12 5 17 7 67
11 5 16 6 68
11 4 15 7 69
11 3 14 8 70
12 3 15 9 71
13 3 16 10 72
13 2 15 11 73
13 1 14 12 74
12 1 13 11 75
11 1 

In [28]:
x1 = 2
y1 = 4

x2 = 3
y2 = 7

u1 = x1 + y1
v1 = abs(x1 - y1)

u2 = x2 + y2
v2 = abs(x2 - y2)


In [30]:
(abs(u1 - u2) + abs(v1 - v2)) / 2

3.0

In [32]:
(u1, v1), (u2, v2)

((6, 2), (10, 4))

In [33]:
abs(x1 - x2) + abs(y1- y2)

4

In [47]:
for j in range(-20, 20):
    print(j & (-j) , j & (~j + 1))

4 4
1 1
2 2
1 1
16 16
1 1
2 2
1 1
4 4
1 1
2 2
1 1
8 8
1 1
2 2
1 1
4 4
1 1
2 2
1 1
0 0
1 1
2 2
1 1
4 4
1 1
2 2
1 1
8 8
1 1
2 2
1 1
4 4
1 1
2 2
1 1
16 16
1 1
2 2
1 1


In [46]:
j = 12
j & -j

4